In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

driver = webdriver.Chrome("./chromedriver")

driver.get("https://www.tripadvisor.com/Tourism-g294197-Seoul-Vacations.html")

# 이름 바꿔서 검색
searchbox = driver.find_element(By.CSS_SELECTOR,"input.qjfqs._G.B-.z._J.Cj.R0")
searchbox.send_keys("Insadong Street")

time.sleep(5)
check=input('일치하는 장소 선택 후 y를 눌러주세요(이외 멈춤)')
if check.upper()=='Y':
    now_url=driver.current_url
url1=now_url.split("-Reviews-")[0]
url2=now_url.split("-Reviews-")[1]

reviews = driver.find_elements(By.CSS_SELECTOR,'div._c')
name=[]
star=[]
content=[]
date=[]
action = ActionChains(driver)
page_num=1
for i in range(1,364): ##################### 531 대신 (영문 리뷰 수)/10 후 반올림 후 +1 한 수 ###################
    reviews = driver.find_elements(By.CSS_SELECTOR,'div._c')
    for review in reviews[1:]: #사진 옆 리뷰 있으면 2부터
        action.move_to_element(review).perform()
        print("y",end="")
        name.append(review.find_element(By.CSS_SELECTOR,'a.BMQDV._F.G-.wSSLS.SwZTJ.FGwzt.ukgoS').text)
        star.append(float(review.find_element(By.CSS_SELECTOR,'svg.UctUV.d.H0').get_attribute("aria-label")[0:3]))
        content.append(review.find_elements(By.CSS_SELECTOR,'span.yCeTE')[1].text.replace("\n"," "))
        date.append(review.find_element(By.CSS_SELECTOR,'.biGQs._P.pZUbB.ncFvv.osNWb').text)
    page_num+=1
    url=url1+"-Reviews-or{}-".format(10*i)+url2
    driver.get(url)
    print("|",end="")
    time.sleep(5)


all=pd.DataFrame([name,star,content,date],index=['이름','별점','리뷰','날짜']).transpose()
print(content)
all.to_csv('Insadong Street_tripadvisor.csv')